<a href="https://colab.research.google.com/github/angelakinoro/KamiLimu-Sentiment_Analysis/blob/Rachael/FineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

IMPORTATIONS


In [88]:
import pandas as pd

!pip install transformers datasets

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from transformers import pipeline
import shutil

!pip install nlpaug

import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as naf
from nlpaug.util import Action


LOADING THE SAMPLE DATA SET

In [89]:
sample = pd.read_csv('/content/synthetic_sample.csv')
sample.head(10)

,id,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,retweets,favorites,is_retweet
0,1395449429801439237,DHOLUO EN DHOWA,NaN,"Respect for all peoples; their Languages, Cult...",2021-02-07 06:14:40,263,2303,2288,False,2021-05-20 18:40:44,@SiambeG @DrMercyKorir 6. FRAUD\n\nRaila Odin...,NaN,Twitter for Android,0.0,0.0,False
1,1393965838756110336,Kenyans.co.ke,"Nairobi, Kenya","Media/News Company based in Nairobi, Kenya.",2009-05-08 20:27:50,371408,72305,892,True,2021-05-16 16:25:29,President Uhuru Kenyatta has launched a number...,NaN,TweetDeck,5.0,17.0,False
2,1393864685716443136,Crispin Okumu 🌐🍍,"Mombasa, KENYA.",You don't inspire others by being perfect. yo...,2013-10-20 18:51:52,11989,1486,10449,False,2021-05-16 09:43:32,@ledamalekina The guy is a project of William ...,NaN,Twitter for Android,0.0,1.0,False
3,1396698645718880256,Solomon Ambuku,"Mombasa, Kenya.",Social Democrat | Governance & Human Rights Cr...,2014-04-23 21:22:36,35964,37551,139746,False,2021-05-24 05:24:41,@citizentvkenya @SamGituku @SpeakerJBMuturi is...,"['Kikuyu', 'MtKenya']",Twitter for Android,0.0,0.0,False
4,1396059409214738432,willys juma,NaN,NaN,2020-12-24 16:58:02,43,524,388,False,2021-05-22 11:04:35,@MarkBichachi Those major projects were initia...,NaN,Twitter for Android,0.0,1.0,False
5,1396022846636412930,Kenya Finest❤,"Nairobi, Kenya",It's Never that Serious 😍• Youth Ambassador• I...,2020-04-07 11:06:18,41557,3782,22950,False,2021-05-22 08:39:18,When Kisumu Youths urged President Uhuru Kenya...,NaN,Twitter for Android,5.0,9.0,False
6,1396020109844664321,Pretty Ammina,Kenya,You Didn't Come All The Way For Nothing Darlin...,2018-04-07 05:26:37,66801,35425,28256,False,2021-05-22 08:28:25,"Police uniforms project\n\nIn July 2020, The N...",NaN,Twitter for Android,13.0,11.0,False
7,1395981700660924417,#SimplicityIsComplex™...,South of Sahara,The more things change the more they remain th...,2020-06-01 17:29:53,4998,4809,33572,False,2021-05-22 05:55:48,Coronation of JB Muturi as Mt. Kenya kingpin i...,NaN,Twitter for Android,0.0,0.0,False
8,1395957601104498690,The Standard Digital,Kenya | Nairobi,"Kenya's bold newspaper, since 1902.",2009-07-02 11:28:51,1546906,164,1971,True,2021-05-22 04:20:02,The Big Four Agenda is part of President Uhuru...,NaN,Hootsuite Inc.,2.0,19.0,False
9,1395767250439872513,Loretta_majesty,Mars,Peace Love and Unity\nStandard Ya Quality sisi...,2020-02-17 06:42:26,815,1275,5545,False,2021-05-21 15:43:39,#UhuruMeansBusiness\nPresident Uhuru Kenyatta ...,['UhuruMeansBusiness'],Twitter for Android,4.0,6.0,False


LABELLING SAMPLE DATA

Bag of Words Algorithm to initially label

In [90]:

# Create Sentiment and Project_Status columns with initial values of 0
sample['Sentiment'] = 0
sample['Project_Status'] = 0

# Remove all columns except 'text'
columns_to_keep = ['text', 'Sentiment', 'Project_Status']
sample = sample[columns_to_keep].copy()

# Sentiment Algorithm (Simple Example)
def analyze_sentiment(text):
    """
    Analyzes the sentiment of a text using a simple keyword approach.
    """
    positive_keywords = ['great', 'amazing', 'excellent', 'happy', 'good', 'love', 'celebrate','nice']
    negative_keywords = ['sad','unhappy','bad', 'terrible', 'awful', 'sad', 'hate', 'poor', 'not', 'but','lies', 'abnormal', 'fraud', 'fraudulent', 'corrupt', 'corruption', 'mpigs']

    text = text.lower()
    for keyword in positive_keywords:
        if keyword in text:
            return 1
    for keyword in negative_keywords:
        if keyword in text:
            return 0
    return 0  # Default to negative if no keywords found

# Apply the sentiment analysis to each row
sample.loc[:,'Sentiment'] = sample['text'].apply(analyze_sentiment)


# Project_Status Algorithm (Example)
def analyze_project_status(text):
    """
    Analyzes the project status based on simple keywords.
    """
    complete_keywords = ['is complete', 'finished', 'done', 'closed']
    incomplete_keywords = ['pending', 'abandon', 'abandoned','fear', 'ongoing', 'in progress', 'delayed', 'failed', 'not', 'bad', 'terrible', 'awful', 'sad', 'hate', 'poor', 'not', 'but','lies', 'abnormal', 'fraud', 'fraudulent', 'corrupt', 'corruption', 'mpigs']

    text = text.lower()
    for keyword in complete_keywords:
        if keyword in text:
            return 1
    for keyword in incomplete_keywords:
        if keyword in text:
            return 0
    return 0  # Default to incomplete if no keywords found

# Apply the project status analysis to each row
sample.loc[:,'Project_Status'] = sample['text'].apply(analyze_project_status)


# Display the updated dataframe
sample[sample['Project_Status'] != 0]

,text,Sentiment,Project_Status
11,Under @NMS_Kenya and president uhuru Kenyatta ...,0,1
17,@StateHouseKenya What u have done yesterday my...,1,1
52,@DonaldBKipkorir President uhuru Kenyatta admi...,0,1
73,We can't generally say President Uhuru Kenyatt...,0,1
101,Allaying fears that president Uhuru Kenyatta's...,0,1
102,Allaying fears that president Uhuru Kenyatta's...,0,1


Labelling the rows manually

In [91]:
# Function to remove irrelevant rows
def remove(df, index):
    df.drop(index, axis=0, inplace=True)  # Correctly pass the index
    return 0

# Function to manually review and edit sentiment and project status
def review_and_edit_data(df):
    """
    Iterates through the dataframe rows, allowing manual review and editing
    of sentiment and project status, without terminating the loop after row removal.
    """
    indices = df.index.tolist()  # Create a list of indices to avoid breaking the iteration

    for index in indices:
        if index not in df.index:  # Skip if the row has been removed
            continue

        row = df.loc[index]  # Access the row by its index
        print(f"\n--- Row {index} ---")
        print("Text:", row['text'])
        print("Current Sentiment:", row['Sentiment'])
        print("Current Project Status:", row['Project_Status'])

        # Get sentiment input
        new_sentiment = input("Enter new Sentiment (0 or 1, leave blank to keep current, enter 2 to remove): ")
        if new_sentiment != "":  # Check if input is not empty
            if int(new_sentiment) == 2:  # Check for removal condition
                remove(df, index)  # Remove the row
                continue  # Skip further input for the removed row
            else:
                df.at[index, 'Sentiment'] = int(new_sentiment)

        # Get project status input
        new_project_status = input("Enter new Project Status (0 or 1, leave blank to keep current, enter 2 to remove): ")
        if new_project_status != "":  # Check if input is not empty
            if int(new_project_status) == 2:  # Check for removal condition
                remove(df, index)  # Remove the row
                continue  # Skip further input for the removed row
            else:
                df.at[index, 'Project_Status'] = int(new_project_status)

    return df
0


# Reviewing and editing a specific range of rows (30 to 70)
sample_to_review = sample.copy()[30:70]
updated_sample = review_and_edit_data(sample_to_review)

# Combine the updated rows with the existing DataFrame
# Here, replace the specific rows in the original DataFrame with the updated ones
sample.update(updated_sample)

# Save the combined DataFrame back to the CSV
sample.to_csv('updated_sample.csv', index=False)

# Display the updated dataframe
print("\nUpdated DataFrame:")
print(sample)



--- Row 30 ---
Text: #MajiMashinani
Thakwe Dam was first proposed in 1953. It is one of the mega projects that President Uhuru Kenyatta… https://t.co/ykp79Ysr55
Current Sentiment: 0
Current Project Status: 0
Enter new Sentiment (0 or 1, leave blank to keep current, enter 2 to remove): 1
Enter new Project Status (0 or 1, leave blank to keep current, enter 2 to remove): 0

--- Row 31 ---
Text: To provide water for irrigation and domestic use, President Uhuru Kenyatta had to embark in dam projects to provide… https://t.co/IKj7eMOL5A
Current Sentiment: 0
Current Project Status: 0
Enter new Sentiment (0 or 1, leave blank to keep current, enter 2 to remove): 1
Enter new Project Status (0 or 1, leave blank to keep current, enter 2 to remove): 0

--- Row 32 ---
Text: President Uhuru Kenyatta commissioned the free water supply project to informal areas #MajiMashinani
Current Sentiment: 0
Current Project Status: 0
Enter new Sentiment (0 or 1, leave blank to keep current, enter 2 to remove): 1
E

In [92]:
import pandas as pd

# Load the existing sample DataFrame from CSV
sample = pd.read_csv('updated_sample.csv')

# Load the existing sample DataFrame from CSV
sample = pd.read_csv('updated_sample.csv')

# Reviewing and editing a specific range of rows (30 to 70)
sample_to_review = sample.copy()[70:100]
updated_sample = review_and_edit_data(sample_to_review)

# Combine the updated rows with the existing DataFrame
# Here, replace the specific rows in the original DataFrame with the updated ones
sample.update(updated_sample)

# Save the combined DataFrame back to the CSV
sample.to_csv('updated_sample.csv', index=False)

# Display the updated dataframe
print("\nUpdated DataFrame:")
print(sample)



--- Row 70 ---
Text: LAPSSET has been Kenya's dream project since 1975 and was given life in 2012. President Uhuru Kenyatta has been a g… https://t.co/pAqddeNrIJ
Current Sentiment: 0
Current Project Status: 0
Enter new Sentiment (0 or 1, leave blank to keep current, enter 2 to remove): 1
Enter new Project Status (0 or 1, leave blank to keep current, enter 2 to remove): 0

--- Row 71 ---
Text: Lamu port is one of the milestone projects by President Uhuru Kenyatta 

#LamuPortRoars https://t.co/ghfqHAPmda
Current Sentiment: 0
Current Project Status: 0
Enter new Sentiment (0 or 1, leave blank to keep current, enter 2 to remove): 1
Enter new Project Status (0 or 1, leave blank to keep current, enter 2 to remove): 0

--- Row 72 ---
Text: President Uhuru Kenyatta today expected to launch the new Lamu port which is part of the LAPSSET project… https://t.co/bAE0SI0Zcc
Current Sentiment: 0
Current Project Status: 0
Enter new Sentiment (0 or 1, leave blank to keep current, enter 2 to remove): 1


In [94]:
# Load the existing sample DataFrame from CSV
sample = pd.read_csv('updated_sample.csv')

# Load the existing sample DataFrame from CSV
sample = pd.read_csv('updated_sample.csv')

# Reviewing and editing a specific range of rows (30 to 70)
sample_to_review = sample.copy()[100:]
updated_sample = review_and_edit_data(sample_to_review)

# Combine the updated rows with the existing DataFrame
# Here, replace the specific rows in the original DataFrame with the updated ones
sample.update(updated_sample)

# Save the combined DataFrame back to the CSV
sample.to_csv('updated_sample.csv', index=False)

# Display the updated dataframe
print("\nUpdated DataFrame:")
print(sample)



--- Row 100 ---
Text: President Uhuru Kenyatta officially opened  Siaya-Bondo water supply and sanitation project.

The objective of the… https://t.co/zJKiSjgXHe
Current Sentiment: 0
Current Project Status: 0
Enter new Sentiment (0 or 1, leave blank to keep current, enter 2 to remove): 1
Enter new Project Status (0 or 1, leave blank to keep current, enter 2 to remove): 

--- Row 101 ---
Text: Allaying fears that president Uhuru Kenyatta's big four projects had been abandoned, Your Yatani has announced an a… https://t.co/Kd7DWy9JuC
Current Sentiment: 0
Current Project Status: 1
Enter new Sentiment (0 or 1, leave blank to keep current, enter 2 to remove): 0
Enter new Project Status (0 or 1, leave blank to keep current, enter 2 to remove): 0

--- Row 102 ---
Text: Allaying fears that president Uhuru Kenyatta's big four projects had been abandoned, Your Yatani has announced an a… https://t.co/vi5Qxf2Ri1
Current Sentiment: 0
Current Project Status: 1
Enter new Sentiment (0 or 1, leave blan

In [10]:
# Reviewing and editing the next 10 rows of the sample dataframe
sample_to_review = sample.copy()[70:90]
updated_sample = review_and_edit_data(sample_to_review)

# Display the updated dataframe
print("\nUpdated DataFrame:")
print(updated_sample)


--- Row 70 ---
Text: LAPSSET has been Kenya's dream project since 1975 and was given life in 2012. President Uhuru Kenyatta has been a g… https://t.co/pAqddeNrIJ
Current Sentiment: 0
Current Project Status: 0
Enter new Sentiment (0 or 1, leave blank to keep current, enter 2 to remove): 1
Enter new Project Status (0 or 1, leave blank to keep current, enter 2 to remove): 0

--- Row 71 ---
Text: Lamu port is one of the milestone projects by President Uhuru Kenyatta 

#LamuPortRoars https://t.co/ghfqHAPmda
Current Sentiment: 0
Current Project Status: 0
Enter new Sentiment (0 or 1, leave blank to keep current, enter 2 to remove): 2

--- Row 72 ---
Text: President Uhuru Kenyatta today expected to launch the new Lamu port which is part of the LAPSSET project… https://t.co/bAE0SI0Zcc
Current Sentiment: 0
Current Project Status: 0
Enter new Sentiment (0 or 1, leave blank to keep current, enter 2 to remove): 1
Enter new Project Status (0 or 1, leave blank to keep current, enter 2 to remove): 0


In [11]:
# Reviewing and editing the next 10 rows of the sample dataframe
sample_to_review = sample.copy()[83:103]
updated_sample = review_and_edit_data(sample_to_review)

# Display the updated dataframe
print("\nUpdated DataFrame:")
print(updated_sample)


--- Row 83 ---
Text: Do we mean BBI was not UHURU KENYATTA project?  It shows that this was not popular initiative but PRESIDENT initiat… https://t.co/jnhvc6SFo3
Current Sentiment: 0
Current Project Status: 0
Enter new Sentiment (0 or 1, leave blank to keep current, enter 2 to remove): 2

--- Row 84 ---
Text: Why are people like Murathe and Tuju breathing fire if the BBI was not UHURU KENYATTA project?  It shows that this… https://t.co/dI3lJtbVU0
Current Sentiment: 0
Current Project Status: 0
Enter new Sentiment (0 or 1, leave blank to keep current, enter 2 to remove): 2

--- Row 85 ---
Text: Uhuru is using Nyanza projects,as wet blanket,to kill raila odingA,detailed programmes,underground,to kill Baba,Fid… https://t.co/QvXzngN4MR
Current Sentiment: 0
Current Project Status: 0
Enter new Sentiment (0 or 1, leave blank to keep current, enter 2 to remove): 
Enter new Project Status (0 or 1, leave blank to keep current, enter 2 to remove): 2
Enter new Sentiment (0 or 1, leave blank to kee

In [12]:
0# Reviewing and editing the next 10 rows of the sample dataframe
sample_to_review = sample.copy()[102:]
updated_sample = review_and_edit_data(sample_to_review)

# Display the updated dataframe
print("\nUpdated DataFrame:")
print(updated_sample)


--- Row 102 ---
Text: Allaying fears that president Uhuru Kenyatta's big four projects had been abandoned, Your Yatani has announced an a… https://t.co/vi5Qxf2Ri1
Current Sentiment: 0
Current Project Status: 1
Enter new Sentiment (0 or 1, leave blank to keep current, enter 2 to remove): 2

--- Row 103 ---
Text: @J_Ladiva @Riverlineridges @PSCharlesHinga @StateHouseKenya #RiverlineRidgesRuiruKE the Project Got a nod from HE t… https://t.co/LcKCZflqiz
Current Sentiment: 0
Current Project Status: 0
Enter new Sentiment (0 or 1, leave blank to keep current, enter 2 to remove): 1
Enter new Project Status (0 or 1, leave blank to keep current, enter 2 to remove): 0

--- Row 104 ---
Text: Uhuru Kenyatta’s deputy was busy going around the country launching projects in an effort that was largely seen as… https://t.co/s7HjfPpzbv
Current Sentiment: 0
Current Project Status: 0
Enter new Sentiment (0 or 1, leave blank to keep current, enter 2 to remove): 0
Enter new Project Status (0 or 1, leave bla

Checking our updated sample


In [95]:
sample[sample['Project_Status'] != 0]


,text,Sentiment,Project_Status
11,Under @NMS_Kenya and president uhuru Kenyatta ...,0,1
17,@StateHouseKenya What u have done yesterday my...,1,1
34,"Originally conceived in 2011, the Lamu corrido...",1,1
50,President Uhuru Kenyatta is making vision 2030...,1,1
58,The LAPSSET project is no longer a dream but a...,1,1
65,"After 9 years of construction, the multi billi...",1,1
73,We can't generally say President Uhuru Kenyatt...,1,1
77,The LAPSSET PROJECT is the most ambitious in K...,1,1
82,Lamu port will be commissioned by President Uh...,1,1
90,@OleItumbi @WilliamsRuto Raila @RailaOdinga La...,1,1


Cleaning out the sample


In [122]:
import re

def clean_text(text):
    """Removes URLs, mentions, hashtags, special characters, converts to lowercase, and removes underscores."""

    # Remove URLs (http, https, www, or t.co links)
    text = re.sub(r'https?://\S+|www\.\S+', '', text)

    # Remove mentions (@username)
    text = re.sub(r'@\w+', '', text)

    # Remove hashtags (#topic)
    text = re.sub(r'#\w+', '', text)

    # Remove special characters and punctuation (except spaces and word characters)
    text = re.sub(r'[^\w\s]', '', text)

    # Remove underscores and convert to lowercase
    text = text.replace('_', ' ').lower()

    return text

# Apply to your DataFrame column
sample['text'] = sample['text'].apply(clean_text)
print(sample)


                                                  text  Sentiment  \
0      6  fraud\n\nraila odinga and kenyatta bbi is...          0   
1    president uhuru kenyatta has launched a number...          0   
2     the guy is a project of william ruto lsk as b...          0   
3       is president uhuru kenyattas pet project no...          0   
4     those major projects were initiated by presid...          0   
..                                                 ...        ...   
128                   muturi is a william ruto project          0   
129  musalia mudavadi is a project of william ruto ...          0   
130  lies these projects are only benefiting jaluos...          0   
131  tanga tanga celebrates everything that hinders...          1   
132  muthaiga square william rutolinked money laund...          0   

     Project_Status  
0                 0  
1                 0  
2                 0  
3                 0  
4                 0  
..              ...  
128              

In [124]:
sample.to_csv('updated_sample.csv', index=False)


GENERATING SYNTHETIC DATA FROM THE LABELLED SAMPLE


In [15]:
import pandas as pd

!pip install transformers datasets

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from transformers import pipeline
import shutil

!pip install nlpaug

import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as naf
from nlpaug.util import Action
from sklearn.utils import shuffle

In [40]:

sample = pd.read_csv('/content/updated_sample (2).csv')


,Project_Status
0,0
1,0
2,0
3,0
4,0
...,...
127,0
128,0
129,0
130,0


In [41]:
# Initialize Augmenters
aug_syn = naw.SynonymAug(aug_src='wordnet')
aug_random_insert = naw.RandomWordAug(action="substitute")  # Using 'substitute' instead of 'insert'

# Function to generate synthetic data
def generate_synthetic_data(df, num_samples=1000):
    """
    Generates synthetic data by applying augmentation techniques.
    Args:
        df (pd.DataFrame): The input DataFrame containing original data.
        num_samples (int): The desired number of synthetic samples to generate.
    Returns:
        pd.DataFrame: A DataFrame containing the generated synthetic data.
    """
    synthetic_data = []
    # Calculate the number of augmentations per original row
    augmentations_per_row = num_samples // len(df) + 1
    for index, row in df.iterrows():
        original_text = row['text']
        sentiment = row['Sentiment']
        project_status = row['Project_Status']

        # Apply augmentations multiple times for each original row
        for _ in range(augmentations_per_row):
            # Apply Synonym Augmentation
            augmented_text_syn = aug_syn.augment(original_text)

            # Apply Random Word Insertion
            augmented_text_random = aug_random_insert.augment(original_text)

            synthetic_data.append({'text': augmented_text_syn, 'Sentiment': sentiment, 'Project_Status': project_status})
            synthetic_data.append({'text': augmented_text_random, 'Sentiment': sentiment, 'Project_Status': project_status})

            # Break if the desired number of samples is reached
            if len(synthetic_data) >= num_samples:
                break
        else:
            # Continue if the inner loop wasn't broken
            continue
        # Inner loop was broken, break the outer loop as well
        break

    return pd.DataFrame(synthetic_data)

# Generate synthetic data from the labeled sample
synthetic_df = generate_synthetic_data(sample, num_samples=1000)

# Concatenate the original and synthetic data
combined_df = pd.concat([sample, synthetic_df], ignore_index=True)

# Shuffle the combined dataframe
combined_df = shuffle(combined_df)

# Save the combined dataframe to a new CSV file
combined_df.to_csv('synthetic_data.csv', index=False)

print("Synthetic data generated and saved to 'synthetic_data.csv'")

Synthetic data generated and saved to 'synthetic_data.csv'


Checking out the synthetic data

In [43]:
# Filter the DataFrame for rows where Project_Status is 0 and Sentiment is 0
filtered_df = combined_df[(combined_df['Project_Status'] == 0) & (combined_df['Sentiment'] == 0)]

# Get the count of rows
num_rows = filtered_df.shape[0]

print(f"Number of rows where Project_Status is 0 and Sentiment is 0: {num_rows}")


Number of rows where Project_Status is 0 and Sentiment is 0: 698


We find that for the majority of the data, when the project is incomplete, the sentiment is likely to be negative.

That's good.
Let's proceed to train


PERFORM TRAIN TEST SPLIT

FINETUNING THE TRAINING SET

In [ ]:
# Assuming 'synthetic_data' is a Pandas DataFrame with columns 'text' and 'Sentiment'

# 1. Prepare the data for fine-tuning
train_data, eval_data = train_test_split(synthetic_data, test_size=0.2)

# Create HuggingFace datasets from pandas DataFrame
train_dataset = Dataset.from_pandas(train_data)
eval_dataset = Dataset.from_pandas(eval_data)

# 2. Choose a pre-trained model and tokenizer
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)  # Assuming binary classification

# 3. Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)  # Change 'text' if the column name is different

train_dataset = train_dataset.map(tokenize_function, batched=True)
eval_dataset = eval_dataset.map(tokenize_function, batched=True)

# 4. Rename the 'Sentiment' column to 'label'
# Ensure the correct column name for your labels (adjust 'Sentiment' if needed)
train_dataset = train_dataset.rename_column("Sentiment", "label")
eval_dataset = eval_dataset.rename_column("Sentiment", "label")

# 5. Specify training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    logging_strategy="steps",  # Log every few steps
    logging_steps=100,  # Adjust as needed to control how often logging occurs
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',  # Ensure logs are saved
    report_to="none",  # Change to 'tensorboard' or 'all' if you want TensorBoard logs
)

# 6. Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

# 7. Fine-tune the model
trainer.train()

# 8. Evaluate the model
eval_results = trainer.evaluate()
print(eval_results)
